## Comparing Linear Regression with Other Models
#### In this file, we are comparing the Linear Regression model with XGBRegressor and Support Vector Regression (SVR) models and evaluate the accuracy. 

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [2]:
# Importing the data from SQLite database into a DataFrame
house_df = pd.read_sql_table('sold_homes', 'sqlite:///sold_homes.db').drop(columns=['index'])
house_df.head()

,#,LSC,EC,St#,Street Name,Abbr,Dir,Municipality,Community,List Price,...,Fam,Kit,Garage Type,A/C,Heat,Contract Date,Sold Date,List Brokerage,Co op Brokerage,MLS#
0,1,Sld,None,38,Scarsdale,Crt,None,Brampton,Brampton W,"$599,000",...,N,1,Built-,Cen,Gas,3/16/2021,3/23/2021,RE/MAX WEST,RE/MAX REALTY,W5153139
1,2,Sld,None,55,Corby,Cres,None,Brampton,Brampton W,"$599,900",...,N,1,None,Cen,Oil,2/23/2021,2/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5125388
2,3,Sld,None,7,Manett,Cres,None,Brampton,Brampton W,"$699,000",...,N,1,Attach,Cen,Gas,1/21/2021,1/26/2021,RE/MAX REAL,HOMELIFE/MIRA,W5091089
3,4,Sld,None,76,Fairglen,Ave,None,Brampton,Brampton W,"$699,500",...,N,1,None,Cen,Gas,3/16/2021,3/17/2021,"EXP REALTY,",ROYALSTARRE,W5153813
4,5,Sld,None,121,Richwood,Cres,None,Brampton,Brampton W,"$699,900",...,N,1,Built-,Cen,Gas,2/21/2021,2/26/2021,RE/MAX REAL,ROYAL LEPAGE,W5123258


In [3]:
# Checking date types of columns
house_df.dtypes

#                   int64
LSC                object
EC                 object
St#                object
Street Name        object
Abbr               object
Dir                object
Municipality       object
Community          object
List Price         object
Sold Price         object
Type               object
Style              object
Br                  int64
Additional          int64
Wr                  int64
Fam                object
Kit                 int64
Garage Type        object
A/C                object
Heat               object
Contract Date      object
Sold Date          object
List Brokerage     object
Co op Brokerage    object
MLS#               object
dtype: object

### Data Preprocessing and Preliminary Features Selection

In [4]:
# Drop the non-beneficial columns
house_df = house_df.drop(["#","LSC","EC","St#","Dir","Municipality","Community","MLS#","Abbr","List Brokerage","Co op Brokerage","Street Name","Heat","A/C"],1)
house_df.head()

,List Price,Sold Price,Type,Style,Br,Additional,Wr,Fam,Kit,Garage Type,Contract Date,Sold Date
0,"$599,000","$681,000",Att/Row/Tw,2-Storey,3,0,2,N,1,Built-,3/16/2021,3/23/2021
1,"$599,900","$767,000",Semi-Detac,Bungalow,3,1,2,N,1,None,2/23/2021,2/26/2021
2,"$699,000","$818,500",Att/Row/Tw,2-Storey,3,1,3,N,1,Attach,1/21/2021,1/26/2021
3,"$699,500","$726,000",Semi-Detac,Bungalow,2,1,2,N,1,None,3/16/2021,3/17/2021
4,"$699,900","$731,000",Att/Row/Tw,2-Storey,3,0,3,N,1,Built-,2/21/2021,2/26/2021


In [5]:
# Converting "List Price" and "Sold Price" to integer type
house_df["List Price"] = house_df["List Price"].replace('[\$,]', '', regex=True).astype(int)
house_df["Sold Price"] = house_df["Sold Price"].replace('[\$,]', '', regex=True).astype(int)
house_df.head()

,List Price,Sold Price,Type,Style,Br,Additional,Wr,Fam,Kit,Garage Type,Contract Date,Sold Date
0,599000,681000,Att/Row/Tw,2-Storey,3,0,2,N,1,Built-,3/16/2021,3/23/2021
1,599900,767000,Semi-Detac,Bungalow,3,1,2,N,1,None,2/23/2021,2/26/2021
2,699000,818500,Att/Row/Tw,2-Storey,3,1,3,N,1,Attach,1/21/2021,1/26/2021
3,699500,726000,Semi-Detac,Bungalow,2,1,2,N,1,None,3/16/2021,3/17/2021
4,699900,731000,Att/Row/Tw,2-Storey,3,0,3,N,1,Built-,2/21/2021,2/26/2021


In [6]:
# Converting the 'Contract Date' and 'Sold Date' to interger type
house_df['Contract Date'] = house_df['Contract Date'].replace('[\/]', '', regex=True).astype(int) 
house_df['Sold Date'] = house_df['Sold Date'].replace('[\/]', '', regex=True).astype(int)
house_df.head()

,List Price,Sold Price,Type,Style,Br,Additional,Wr,Fam,Kit,Garage Type,Contract Date,Sold Date
0,599000,681000,Att/Row/Tw,2-Storey,3,0,2,N,1,Built-,3162021,3232021
1,599900,767000,Semi-Detac,Bungalow,3,1,2,N,1,None,2232021,2262021
2,699000,818500,Att/Row/Tw,2-Storey,3,1,3,N,1,Attach,1212021,1262021
3,699500,726000,Semi-Detac,Bungalow,2,1,2,N,1,None,3162021,3172021
4,699900,731000,Att/Row/Tw,2-Storey,3,0,3,N,1,Built-,2212021,2262021


In [7]:
house_df.dtypes

List Price        int32
Sold Price        int32
Type             object
Style            object
Br                int64
Additional        int64
Wr                int64
Fam              object
Kit               int64
Garage Type      object
Contract Date     int32
Sold Date         int32
dtype: object

In [8]:
# Determine the number of unique values in each column
# Since the values are less than 10, no bucketing is required
house_cat = house_df.dtypes[house_df.dtypes == "object"].index.tolist()
house_df[house_cat].nunique()

Type            5
Style          10
Fam             2
Garage Type     6
dtype: int64

In [9]:
# Look at "Style"value counts for binning
style_counts = house_df["Style"].value_counts()
style_counts

2-Storey      847
Bungalow      100
Backsplit      88
3-Storey       58
Bungalow-R     52
Sidesplit      34
1 1/2 Stor      5
Other           3
Bungaloft       3
2 1/2 Stor      3
Name: Style, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ...?
replace_style = list(style_counts[style_counts < 30].index)

# Replace in dataframe
for app in replace_style:
    house_df.Style = house_df.Style.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.Style.value_counts()

2-Storey      847
Bungalow      100
Backsplit      88
3-Storey       58
Bungalow-R     52
Sidesplit      34
Other          14
Name: Style, dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(house_df[house_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(house_cat)
encode_df.head()

,Type_Att/Row/Tw,Type_Detached,Type_Link,Type_Semi-Detac,Type_Vacant Lan,Style_2-Storey,Style_3-Storey,Style_Backsplit,Style_Bungalow,Style_Bungalow-R,Style_Other,Style_Sidesplit,Fam_N,Fam_Y,Garage Type_Attach,Garage Type_Built-,Garage Type_Carpor,Garage Type_Detach,Garage Type_None,Garage Type_Other
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
house_df = house_df.merge(encode_df,left_index=True, right_index=True)
house_df = house_df.drop(house_cat,1)
house_df.head()

,List Price,Sold Price,Br,Additional,Wr,Kit,Contract Date,Sold Date,Type_Att/Row/Tw,Type_Detached,...,Style_Other,Style_Sidesplit,Fam_N,Fam_Y,Garage Type_Attach,Garage Type_Built-,Garage Type_Carpor,Garage Type_Detach,Garage Type_None,Garage Type_Other
0,599000,681000,3,0,2,1,3162021,3232021,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,599900,767000,3,1,2,1,2232021,2262021,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,699000,818500,3,1,3,1,1212021,1262021,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,699500,726000,2,1,2,1,3162021,3172021,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,699900,731000,3,0,3,1,2212021,2262021,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
# Creating copy of Dataframe to be used later in other models
house1_df = house_df.copy()

## Multiple Linear Regression Model 
Here we creating Multiple Linear Regression machine learning model using LinearRegression() from sklearn. Multiple linear regression is used to estimates the relationship between several independent variables (features) and one dependent variable.

In [14]:
# Split our preprocessed data into our features and target arrays
y = house_df["Sold Price"].values
X = house_df.drop(columns=["Sold Price"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,train_size=0.8,random_state=5)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Create a model with scikit-learn and fit the data in the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [17]:
# The model creates predicted y values based on X values
y_pred = model.predict(X_test_scaled)

In [18]:
# Calculating R2 value or Coefficient of Determination
test_set_r2 = r2_score(y_test, y_pred)
print("Coefficient of Determination:",test_set_r2)

Coefficient of Determination: 0.9536209250631626


In [19]:
# Calculating MAE or Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:",mae)

Mean Absolute Error: 51510.07773405633


In [20]:
# Calculating RMSE or Root Mean Square Error
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
print("Root Mean Squared Error:",rmse)

Root Mean Squared Error: 69723.40186718859


## XGBRegressor Model
We are using XGBRegressor model from XGBoost library (Extreme Gradient Boosting). It is an implementation of gradient boosting trees algorithm. We are comparing the XGBRegressor model with Liner Regression model to see if there is an improvement in performance. 

In [21]:
# Split our preprocessed data into our features and target arrays
y1 = house1_df["Sold Price"].values
X1 = house1_df.drop(columns=["Sold Price"]).values

# Split the preprocessed data into a training and testing dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1,test_size=0.2,train_size=0.8,random_state=5)

In [22]:
# Create a StandardScaler instances
scaler1 = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler1.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

In [23]:
import xgboost
# Creating an instance of the XGBRegressor
model1 = xgboost.XGBRegressor()

In [24]:
# Fitting the training data to the model
model1.fit(X1_train_scaled,y1_train)

[14:36:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [25]:
# Obtaining the predictions for the testing data 
y1_pred = model1.predict(X1_test_scaled)

In [26]:
# Calculating R2 value or Coefficient of Determination
test_set2_r2 = r2_score(y1_test, y1_pred)
print("Coefficient of Determination:",test_set2_r2)

Coefficient of Determination: 0.9213758427869377


In [27]:
# Calculating MAE or Mean Absolute Error
mae1 = mean_absolute_error(y1_test, y1_pred)
print("Mean Absolute Error:",mae1)

Mean Absolute Error: 50919.912133891215


In [28]:
# Calculating RMSE or Root Mean Square Error
rmse1 = (np.sqrt(mean_squared_error(y1_test, y1_pred)))
print("Root Mean Squared Error:",rmse1)

Root Mean Squared Error: 90781.11846120015


#### Comparison: Here we can see that though the MAE has improved (compared with Linear Regression) from 51510.07 to 50919.91, the RMSE has increase from 69723.40 to 90781.11 which is quite significant. Also, R2 vlaue has also gone down from 0.95 to 0.92. Therefore, it is concluded that Linear Regression works better than XGBRegressor for this dataset.

## Support Vector Regression (SVR)
SVR is a regression model in which we try to fit the error in a certain threshold (unlike minimizing the error rate we were doing in the previous cases). SVR can work for linear as well as non-linear problems depending on the kernel we choose. 

In [29]:
# Split our preprocessed data into our features and target arrays
y2 = house1_df["Sold Price"].values
X2 = house1_df.drop(columns=["Sold Price"]).values

# Split the preprocessed data into a training and testing dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,test_size=0.2,train_size=0.8,random_state=5)

In [30]:
# Create a StandardScaler instances
scaler2 = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler2.fit(X1_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

In [31]:
from sklearn.svm import SVR
# Creating an instance of the SVM
model2 = SVR(kernel='rbf')

In [32]:
# Fitting the training data to the model
model2.fit(X2_train_scaled, y2_train)

SVR()

In [33]:
y2_pred = model2.predict(X2_test_scaled)

In [34]:
# Calculating R2 value or Coefficient of Determination
test_set3_r2 = r2_score(y2_test, y2_pred)
print("Coefficient of Determination:",test_set3_r2)

Coefficient of Determination: -0.10207696260136578


In [35]:
# Calculating MAE or Mean Absolute Error
mae2 = mean_absolute_error(y2_test, y2_pred)
print("Mean Absolute Error:",mae1)

Mean Absolute Error: 50919.912133891215


In [36]:
# Calculating RMSE or Root Mean Square Error
rmse2 = (np.sqrt(mean_squared_error(y2_test, y2_pred)))
print("Root Mean Squared Error:",rmse1)

Root Mean Squared Error: 90781.11846120015


#### Comparison: Here we can see that though the MAE has improved (compared with Linear Regression) from 51510.07 to 50919.91, the RMSE has increase from 69723.40 to 90781.11 which is quite significant. Also, R2 vlaue has also deteriorated to -0.10. Therefore, it is concluded that Linear Regression works better than XGBRegressor for this dataset.